# Intervals
**Purpose:** Show ways of making operations in Python on sets defined as lists of intervals e.g. sets of the form A=[[a1, a2], [a3, a4], ..., [an-1, an]] and B=[[b1, b2], [b3, b4], ..., [bm-1, bm]].

## Test of Intersection

Union of two sets A=[a1, a2] and B=[b1, b2] can be done by first checking that the sets intersect a check of this can be done using below rule:

$$
\begin{aligned}
\left[a1, a2\right] \cap \left[b1, b2\right] = \emptyset &\iff a2 \lt b1 \lor b2 \lt a1 \\
&\Updownarrow \\
\left[a1, a2\right] \cap \left[b1, b2\right] \neq \emptyset &\iff a2 \geq b1 \land b2 \geq a1
\end{aligned}
$$

Using negation and De Morgan's law.

## Union
To union a list of sets one can use below.

In [210]:
from collections import defaultdict
from typing import List

ListOfIntervals = List[List[int]]

A = [[1, 3], [6, 8], [9, 10], [10, 10], [11, 12], [11, 12]]
B = [[2, 4], [7, 8], [9, 11], [10, 10], [14, 16]]

def reduce(I: ListOfIntervals, dialate: int = 1) -> ListOfIntervals:
    """Reduces a list of intervals to a compressed form.
    
    Args:
        dialate (int): An integer that dialates all intervals before they're reduced.
            The dialation is reversed afterwards. Can be used if one does not care
            about short differences.
            
    Returns:
        ListOfIntervals: A compressed list of intervals without overlapping intervals.
    """
    X = defaultdict(list)
    for t1, t2 in I:
        X[t1 - dialate].append("start")
        X[t2 + dialate].append("stop")

    union = []
    start = []
    stop = []
    for key in sorted(X.keys()):
        for value in X[key]:
            if value == "start":
                start.append(key)
            else:
                stop.append(key)

        if len(start) == len(stop):  # All intervals are closed.
            union.append([min(start) + dialate, max(stop) - dialate])
            start = []
            stop = []
            
    return union

print(reduce(A))
print(reduce(B))
print(reduce(A + B))  # union

[[1, 3], [6, 12]]
[[2, 4], [7, 11], [14, 16]]
[[1, 16]]
